# Домашняя работа по NLP №1

Для начала загрузим все нужные модули:

In [287]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from nltk import word_tokenize
from collections import Counter
from pymystem3 import Mystem
from string import punctuation

m = Mystem()
punctuation += "\xa0—\xa0\n«»"

*tqdm* я загрузил для себя, он необязателен для выполнения задания.

Закачиваем данные — последние отзывы из Кинопоиска на различные фильмы:

In [259]:
# у нас будут четыре списка:
# два для обработки и два для
# тестирования.
good_reviews = []
bad_reviews = []
good_reviews_test = []
bad_reviews_test = []

# цикл, создающий эти четыре списка
for i in tqdm(range(25)):
    
    # при помощи requests закачиваем по странице.
    # у кинопоиска есть разделы "плохие отзывы" и
    # "хорошие отзывы", что нам сильно помогает при парсинге.
    good_reviews_page = requests.get(f"https://www.kinopoisk.ru/reviews/status/good/page/{str(i+1)}/#list")
    bad_reviews_page = requests.get(f"https://www.kinopoisk.ru/reviews/status/bad/page/{str(i+1)}/#list")
    # варим суп из полученного html-а.
    good_soup = BeautifulSoup(good_reviews_page.text, "html.parser")
    bad_soup = BeautifulSoup(bad_reviews_page.text, "html.parser")
    
    # тексты отзыва лежат внутри тэга div
    # класса brand_words.
    good_reviews_html = good_soup.find_all("div", {"class": "brand_words"})
    bad_reviews_html = bad_soup.find_all("div", {"class": "brand_words"})
    
    # первые 20 страниц отзывов возьмем для
    # создания нашей модели, оставшиеся 5 —
    # для тестирования
    if i < 20:
        # при помощи MyStem лемматизируем
        # текст и выбрасываем все токены-знаки
        # препинания. на выходе получаем список
        # всех лемм.
        for review in good_reviews_html:
            lemmas = m.lemmatize(review.get_text())
            good_reviews.extend([lemma for lemma in lemmas if not lemma.strip() in punctuation])
        for review in bad_reviews_html:
            lemmas = m.lemmatize(review.get_text())
            bad_reviews.extend([lemma for lemma in lemmas if not lemma.strip() in punctuation])
    else:
        # здесь делаем то же самое, но нам
        # вместо extend нужен append, т.к.
        # мы рассматриваем отзывы
        # по отдельности.
        for review in good_reviews_html:
            lemmas = m.lemmatize(review.get_text())
            good_reviews_test.append([lemma for lemma in lemmas if not lemma.strip() in punctuation])
        for review in bad_reviews_html:
            lemmas = m.lemmatize(review.get_text())
            bad_reviews_test.append([lemma for lemma in lemmas if not lemma.strip() in punctuation])
            

Получаем четыре списка, содержимое которых выглядит примерно вот так:

In [280]:
bad_reviews[:10]

['история',
 'ничто',
 'не',
 'учить',
 'даже',
 'хороший',
 'мир',
 'сей',
 'кастинг',
 'это']

In [278]:
bad_reviews_test[0][:10]

['после',
 'просмотр',
 'данный',
 'фильм',
 'я',
 'становиться',
 'понятно',
 'что',
 'ничто',
 'не']

Теперь создадим функцию, которая создаст для нас два нужных нам множества.

In [281]:
# на входе: два "корпуса" слов.
def unique_words(corpus1, corpus2):
    # множества слов, уникальных для
    # положительных/отрицательных отзывов.
    corpus1_unique = set()
    corpus2_unique = set()
    
    # счетчики, нужные для того, чтобы
    # отбросить слова с низкой частотностью.
    corpus1_cnt = Counter(corpus1)
    corpus2_cnt = Counter(corpus2)
    
    # из счетчиков создаем сеты, в которых
    # оставляем только те слова, которые
    # встречаются больше пяти раз.
    corpus1_tokens = set([k for k,v in corpus1_cnt.items() if v > 5])
    corpus2_tokens = set([k for k,v in corpus2_cnt.items() if v > 5])
    
    # проходим по двум сетам, чтобы
    # пополнить сеты уникальных слов.
    for token in corpus1_tokens:
        if token not in corpus2_tokens:
            corpus1_unique.add(token)
    for token in corpus2_tokens:
        if token not in corpus1_tokens:
            corpus2_unique.add(token)
    
    # на выходе: кортеж из двух
    # множеств уникальных слов.
    return (corpus1_unique, corpus2_unique)

Применяем функцию на двух "словарях":

In [282]:
unique = unique_words(good_reviews, bad_reviews)

Напишем функцию, которая посчитает в тексте (списке лемм) кол-во "положительных" слов и предположит его тональность.

In [284]:
# на входе: отзыв-набор лемм.
def review_tone(review):
    positive = 0
    negative = 0
    # для каждой леммы смотрим, есть
    # ли она в одном из множеств. если
    # есть, то прибавляем 1 к одной из
    # переменных.
    for l in review:
        if l in unique[0]:
            positive += 1
        elif l in unique[1]:
            negative += 1
    # есть случаи, когда они равны
    # нулю или равны друг другу.
    # тогда считаем, что наш код не
    # смог определить тональность.
    if positive - negative == 0:
        return None
    # в остальных случаях все зависит
    # от того, каких слов больше.
    if positive < negative:
        return "negative"
    return "positive"

Посчитаем точность нашей модели:

In [286]:
# количество верных определений.
correct = 0

# количество случаев, когда машина
# не нашла ответа.
unknown = 0

# для каждого из тестовых наборов
# пишем такие циклы:
for review in good_reviews_test:
    if review_tone(review) == "positive":
        correct += 1
    elif not review_tone(review):
        unknown += 1
for review in bad_reviews_test:
    if review_tone(review) == "negative":
        correct += 1
    elif not review_tone(review):
        unknown += 1

# результат теста:
print("accuracy:")
print(correct/((len(good_reviews_test)+len(bad_reviews_test)) - unknown))

accuracy:
0.7216494845360825


Способы улучшить модель:
* Во время тестирования брать из отзыва новые уникальные слова для первого сета и обновлять второй сет.
* Выбрасывать уникальные слова, если они несколько раз встречаются в одном и том же отзыве и больше не встречаются нигде (в этом случае нам придется использовать не extend при сборе данных, а append). Параметры можно менять.
* Учитывать частотности уникальных слов и не просто прибавлять по единице к positive и negative с каждым уникальным словом, а умножать ее на соответствующий коэффициент и только потом прибавлять.

*Миша Сонькин*